In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
DIR = "/kaggle/input/"

> Main Idea:
 - get a BERT/transformer model to classify on the emotions (this might add some knowledge to the model for the actual task)
 - get another BERT model to do something similar to entity extraction with the tweet and emotion as the input (cureently did QA BERT)

In [ ]:
df = pd.read_csv(DIR+"tweet-sentiment-extraction/train.csv")
df.dropna(inplace=True)
df

In [ ]:
text = df['text'].values
selected_text = df['selected_text'].values
sentiment = df['sentiment'].values

In [ ]:
MODEL_NAME = DIR+"qa-model3"#"twittersentimentqamodel1" #"bert-base-uncased"

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME) #, do_lower_case=True)

In [ ]:
import torch

In [ ]:
from transformers import BertForQuestionAnswering, BertConfig, AdamW

model = BertForQuestionAnswering.from_pretrained(MODEL_NAME, output_attentions=False, output_hidden_states=False)

model.cuda()

In [ ]:
df_test = pd.read_csv(DIR+"tweet-sentiment-extraction/test.csv")#("../input/tweet-sentiment-extraction/test.csv")

print(df_test['text'].isnull().sum())
print(df_test['sentiment'].isnull().sum())

df_test.dropna(axis = 0, how ='any',inplace=True) ;

text = df_test['text'].apply(str)
sentiment = df_test['sentiment'].apply(str)

text = text.values
print(text[0])
sentiment = sentiment.values
print(sentiment[0])

In [ ]:
import torch

input_ids = []
attention_masks = []
token_type_ids = []

for i in range(len(text)):
    encoded = tokenizer.encode_plus(
      sentiment[i],
      text[i],
      add_special_tokens=True,
      max_length=150,
      pad_to_max_length=True,
      return_token_type_ids=True,
      return_attention_mask=True,
      return_tensors='pt'
    )
    
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])
    token_type_ids.append(encoded['token_type_ids'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
token_type_ids = torch.cat(token_type_ids, dim=0)

print(input_ids[0])
print(attention_masks[0])
print(token_type_ids[0])

print(len(input_ids[0]))

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch.utils.data import TensorDataset, random_split

testset = TensorDataset(input_ids, attention_masks, token_type_ids)
batch_size = 64

testloader = DataLoader(testset,
                      batch_size,
                      sampler=SequentialSampler(testset))

In [ ]:
df_test["selected_text"] = df_test["text"]

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

df_submit = pd.read_csv(DIR+"tweet-sentiment-extraction/sample_submission.csv")#("../input/tweet-sentiment-extraction/sample_submission.csv")
def test():
    key = 0
    model.eval()

    for step, data in enumerate(testloader):

        input_id = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_id = data[2].to(device)

        with torch.no_grad():
            start_scores, end_scores = model(input_id, attention_mask, token_type_id)
        
        series_submit = pd.Series()    
        for i in range(input_id.shape[0]):
            all_tokens = tokenizer.convert_ids_to_tokens(input_id[i])
            answer = ' '.join(all_tokens[torch.argmax(start_scores[i]) : torch.argmax(end_scores[i])+1])
            print(answer)
        
#             answer = answer.detach()
#             series_submit = pd.Series(answer)
            df_test['selected_text'][key] = answer
            key += 1

test()

In [ ]:
df_test

In [ ]:
sub = df_test
sub

In [ ]:
def find_all(a_str, sub, start = 0):
  while start <= len(a_str):
    start = a_str.lower().find(sub, start)
    #print(start)
    if start == -1: return
    yield start
    start += 1 #len(sub) # use start += 1 to find overlapping matches

In [ ]:
selected_text_list = []
counter = 0
for index, row in sub.iterrows():
  txt_lst = str(row["selected_text"]).split(" ")
  for i, token in enumerate(txt_lst):
    if token[:2] == "##":
      txt_lst[i] = token[2:]
    
    
  start = row["text"].lower().find(txt_lst[0])
  last_word = txt_lst[-1]
  end = row["text"].lower().rfind(last_word) + len(last_word)
  final_text = row["text"][start:end]
  '''print("XXX ", final_text)'''
  
  '''while len(final_text.replace(" ", "")) > len(''.join(txt_lst)):
    #counter +=1
    start = final_text.lower().find(txt_lst[0], start+1)
    end = final_text.lower().rfind(last_word, start-1 + len(last_word)) + len(last_word)
    final_text = final_text[start:end]
    print("XXX ", final_text)'''
  if len(final_text.replace(" ", "")) != len(''.join(txt_lst)):
    counter +=1
    print(row["text"])
    print(txt_lst)
    print(start)
    print(end)
    print(final_text)
    print("-----")

  selected_text_list.append(final_text)

print(counter)

In [ ]:
"""selected_text_list = []
counter = 0
for index, row in sub.iterrows():
  txt_lst = row["selected_text"].split(" ")
  txt_lst_short = []
  for i, token in enumerate(txt_lst):
    if token[:2] == "##":
      txt_lst[i] = token[2:]

  
  start_list = list(find_all(row["text"], txt_lst[0]))
  start = start_list[0]
  
  if len(start_list) > 1 and len(txt_lst) > 1:
    second_list = list(find_all(row["text"], txt_lst[1], start_list[0]+len(txt_lst[0])))
    #print(second_list)
    for start_item in start_list:
      #print(start_item)
      if second_list[0] == start_item + len(txt_lst[0]) or second_list[0] == start_item + len(txt_lst[0]) + 1:
        #print("selected!")
        start = start_item
        break


  last_word = txt_lst[-1]
  #end = row["text"].lower().rfind(last_word) + len(last_word)
  #end = row["text"].rfind(last_word) + len(last_word)
  
  end_list = list(find_all(row["text"], txt_lst[-1]))
  end = end_list[-1] + len(last_word)
  '''if len(end_list) > 1 and len(txt_lst) > 1:
    secondlast_list = list(find_all(row["text"], txt_lst[-2], start + len(txt_lst[0])))
    for end_item in end_list:
      if len(secondlast_list) > 0 and (end_item + len(last_word) > start):
        if secondlast_list[-1] + len(txt_lst[-2]) == end_item or secondlast_list[-1] + len(txt_lst[-2]) + 1 == end_item:
          end = end_item + len(last_word)
          break'''


  final_text = row["text"][start:end]

  #while len(final_text.lower().replace(" ", "")) > len(''.join(txt_lst)):
  #  end = end_list[end_list.index(end) - 1]
    

  #if final_text.lower().replace(" ", "") != ''.join(txt_lst):
  #  final_text = " ".join(txt_lst)


  if final_text.lower().replace(" ", "") != ''.join(txt_lst):
    counter +=1
    print("start_list: ", start_list)
    print("end_list: ", end_list)
    print(row["text"])
    print(txt_lst)
    print(start)
    print(end)
    print(final_text)
    print("-----")

  selected_text_list.append(final_text)

#print(counter)"""

In [ ]:
print(counter)

In [ ]:
sub["selected_text"] = selected_text_list
sub.drop(columns=["text", "sentiment"], inplace = True)
sub

In [ ]:
sub.to_csv("submission.csv", index=False)